## Tool Use with Claude

This notebook walks through a simple implementation of function calling and tool use with Claude. The tools Claude has access to in this demo are defined in the tools.py file.

### Imports and Configuration
First we'll import libraries and load environment variables. Make sure to set your Anthropic API key.

In [41]:
%pip install anthropic
%pip install requests


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [42]:
import sys
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
import os
from typing import Any
import tools
import dotenv
import json

dotenv.load_dotenv()
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
ANTHROPIC_API_KEY=os.environ.get('ANTHROPIC_API_KEY')

### Tool use prompt

Now, let's create a function that creates the tool use prompt that Claude will use. This will take in a list of our tool descriptions (as defined in tools.py) and the question asked by the user.

In [43]:
def get_system_prompt(tools_string):
    json_sample_call = {
        "function_calls": [
            {
                "tool_name": "$TOOL_NAME",
                "parameters": {
                    "$PARAMETER_NAME": "$PARAMETER_VALUE"
                }
            }
        ]
    }

    return f"""
In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:

<function_calls>
{json.dumps(json_sample_call)}
</function_calls>

Here are the tools available:
{tools_string}
"""

def get_initial_messages(user_input):
    messages = [{ 'role': 'user', 'content': user_input}]

    return messages

### Add tools

This function will add our tool specs we create in tools.py to the tool use prompt.

In [44]:
def add_tools():
    # tools_string = ""
    # for tool_spec in tools.list_of_tools_specs:
    #     tools_string += tool_spec
    return json.dumps(tools.list_of_tools_specs_json)

### Call function

This function will call the actual function we define in tools.py.

In [45]:
def call_function(tool_name, parameters):
    func = getattr(tools, tool_name)
    output = func(**parameters)
    return output

### Add result

This function will format the result from our function call into the proper JSON dictionary. We're enclosing the dictionary in XML tags so Claude can better understand the input.

In [46]:
def format_result(tool_name, output):
    out = {
        "results": [{
            "tool_name": tool_name,
            "stdout": output
        }]
    }
    
    return f"<function_results>{json.dumps(out)}</function_results>"

### Run loop

This function is the actual orchestrator of the function calling logic. Here's how it works:

1. We kick off a loop that first calls Claude with our prompt. This prompt will have the tool specs and the user input loaded into it.
2. We get the completion from Claude and check if the stop sequence for the completion was the closing XML tag for a function call, ```</function_calls>```
3. If the completion does in fact contain a function call, we extract out the tool name and we parse the tool parameters from the JSON input.
4. We then call the function that Claude has decided to invoke.
5. We take the results of the function call, format them into a JSON dictionary, and add them back to the prompt.
6. We repeat the loop starting at step 1 with the original prompt plus the text that has been appended.
7. This process continues until Claude finally outputs an answer and we break the loop.

In [47]:
anthropic = Anthropic(api_key=ANTHROPIC_API_KEY)
def run_loop(system_prompt, initial_messages):
    # Start function calling loop

    messages = initial_messages.copy()
    while True:
        # Get a completion from Claude
        print(f'[messages] {messages}')

        partial_completion = anthropic.beta.messages.create(messages=messages,
                                                        stop_sequences=["</function_calls>"],
                                                        model="claude-2.1",
                                                        system=system_prompt,
                                                        max_tokens=1000,
                                                        temperature=0)
        partial_completion, stop_reason, stop_seq = partial_completion.content[0], partial_completion.stop_reason, partial_completion.stop_sequence # type: ignore

        content = partial_completion.text
        print(f'[content] {content}')
        if stop_reason == 'stop_sequence' and stop_seq == '</function_calls>':
            # If Claude made a function call

            # Find function call
            tag = "<function_calls>"
            tools_string_index = content.find(tag)
            if tools_string_index == -1:
                print("Unable to parse function call, invocation tag '{tag}' not found.")
                break

            tools_string = content[tools_string_index+len(tag):]

            # Convert to dictionary and check that Claude provided all the required parameters
            tools_to_invoke = json.loads(tools_string)

            if 'function_calls' not in tools_to_invoke:
                print("Unable to parse function call, invalid JSON or missing 'function_calls' key")
                break
        
            if not isinstance(tools_to_invoke['function_calls'], list):
                print("Unable to parse function call, invalid JSON or missing list of function calls")
                break

            function_call = tools_to_invoke['function_calls'][0]
            if 'tool_name' not in function_call or 'parameters' not in function_call:
                print("Unable to parse function call, missing tool name or parameters")
                break

            # Invoke the function
            output = call_function(**function_call)

            # Put the stop sequence back in the content
            content += '</function_calls>'
            function_result = format_result(function_call['tool_name'], output)
            
            # Append the results to the assistant prompt
            content += function_result

            # Modify the sequence by appending the assistant role to the Messages call
            messages = initial_messages.copy()
            messages.append({ 'role': 'assistant', 'content': content})
        else:
            # If Claude did not make a function call
            # outputted answer
            print(f'-------------------- Result --------------------')
            print(partial_completion.text)
            break

In [48]:
user_input = "Can you check the weather for me in Oakland, CA?"
tools_string = add_tools()
system_prompt = get_system_prompt(tools_string)
messages = get_initial_messages(user_input)
run_loop(system_prompt, messages)

-------------------- Step 1 --------------------
[messages] [{'role': 'user', 'content': 'Can you check the weather for me in Oakland, CA?'}]


[content] <function_calls>
{"function_calls": [{"tool_name": "get_lat_long", "parameters": {"place": "Oakland, CA"}}]}

-------------------- Step 2 --------------------
[messages] [{'role': 'user', 'content': 'Can you check the weather for me in Oakland, CA?'}, {'role': 'assistant', 'content': '<function_calls>\n{"function_calls": [{"tool_name": "get_lat_long", "parameters": {"place": "Oakland, CA"}}]}\n</function_calls><function_results>{"results": [{"tool_name": "get_lat_long", "stdout": {"latitude": "37.8044557", "longitude": "-122.271356"}}]}</function_results>'}]
[content] 

To get the weather data for Oakland, I first called the get_lat_long tool to convert the place name "Oakland, CA" into latitude and longitude coordinates. The tool returned 37.8044557, -122.271356 as the coordinates.

Now I can call the get_weather tool to get weather data for those coordinates:

<function_calls>
{"function_calls": [{"tool_name": "get_weather", "parameters": {"latitude": "37.8044557", "longitu